Copyright 2024 - Forusone : shins777@gmail.com

Licensed under the Apache License, Version 2.0 (the "License");
you may not use this file except in compliance with the License.
You may obtain a copy of the License at

   https://www.apache.org/licenses/LICENSE-2.0

Unless required by applicable law or agreed to in writing, software
distributed under the License is distributed on an "AS IS" BASIS,
WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
See the License for the specific language governing permissions and
limitations under the License.

# Build Cloud Function serving advanced RAG archiectures.


## Set working directory and mount Google drive

In [1]:
#  For only colab
import sys

if "google.colab" in sys.modules:
    from google.colab import auth
    auth.authenticate_user()

    # To access contents in Google drive
    from google.colab import drive
    drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
#  source root directory
src_dir = '/content/drive/MyDrive/projects/genai_workshop/notebook/03.cloud function/functions'

# cloud function src directory.
import os
os.chdir(src_dir)
!pwd

/content/drive/MyDrive/projects/genai_workshop/notebook/03.cloud function/functions


## Source code for Cloud function

In [4]:
from google.colab import files
files.view(f'{src_dir}/main.py')
files.view(f'{src_dir}/requirements.txt')
files.view(f'{src_dir}/rag/constant.py')
files.view(f'{src_dir}/rag/controller.py')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
#%%writefile table_search.py

Writing table_search.py


## Authentication

In [5]:
!gcloud auth application-default login
!gcloud config set project ai-hangsik
!gcloud auth application-default set-quota-project ai-hangsik


You are running on a Google Compute Engine virtual machine.
The service credentials associated with this virtual machine
will automatically be used by Application Default
Credentials, so it is not necessary to use this command.

If you decide to proceed anyway, your user credentials may be visible
to others with access to this virtual machine. Are you sure you want
to authenticate with your personal account?

Do you want to continue (Y/n)?  y

Go to the following link in your browser, and complete the sign-in prompts:

    https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=764086051850-6qr4p6gpi6hn506pt8ejuq83di341hur.apps.googleusercontent.com&redirect_uri=https%3A%2F%2Fsdk.cloud.google.com%2Fapplicationdefaultauthcode.html&scope=openid+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fuserinfo.email+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcloud-platform+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fsqlservice.login&state=arD0kqSXdTQVNE1k7nHdGg81M4fWqY&prompt=consent&token_

In [6]:
!pwd

/content/drive/MyDrive/projects/genai_workshop/notebook/03.cloud function/functions


## Deploy Cloud Run - Text Processing

In [15]:
!gcloud functions deploy it_search --allow-unauthenticated \
	--region=asia-northeast3 \
	--gen2 \
	--memory=8Gi \
	--cpu=4 \
	--concurrency=4 \
	--min-instances=2 \
	--max-instances=5 \
	--timeout=30s \
	--runtime=python312 \
  --trigger-http \
	--source=. \
	--entry-point=search


  [INFO] A new revision will be deployed serving with 100% traffic.
You can view your function in the Cloud Console here: https://console.cloud.google.com/functions/details/asia-northeast3/it_search?project=ai-hangsik

buildConfig:
  automaticUpdatePolicy: {}
  build: projects/721521243942/locations/asia-northeast3/builds/5947b816-fa6f-492d-af18-e7976a802ecb
  dockerRegistry: ARTIFACT_REGISTRY
  dockerRepository: projects/ai-hangsik/locations/asia-northeast3/repositories/gcf-artifacts
  entryPoint: search
  runtime: python312
  source:
    storageSource:
      bucket: gcf-v2-sources-721521243942-asia-northeast3
      generation: '1724760315825580'
      object: it_search/function-source.zip
  sourceProvenance:
    resolvedStorageSource:
      bucket: gcf-v2-sources-721521243942-asia-northeast3
      generation: '1724760315825580'
      object: it_search/function-source.zip
createTime: '2024-08-19T18:57:47.726297631Z'
environment: GEN_2
labels:
  deployment-tool: cli-gcloud
name: projec

## REST API Call to Cloud function.

In [16]:
!curl -X POST https://it-search-721521243942.asia-northeast3.run.app \
-H 'Content-Type: application/json' \
-d '{"question":"개인정보 보호법에 대해서 설명해주세요."}'



{"result": ["## 개인정보 보호법에 대한 설명\n\n개인정보 보호법은 개인정보의 수집, 이용, 제공 등 처리 과정에서 정보주체의 권익을 보호하고 개인정보를 안전하게 관리하기 위한 법률입니다. \n\n**주요 내용은 다음과 같습니다.**\n\n* **개인정보 수집 및 이용**: 개인정보처리자는 정보주체의 동의를 받거나 법률에 특별한 규정이 있는 경우에만 개인정보를 수집하고, 수집 목적 범위 내에서만 이용할 수 있습니다.\n* **개인정보 수집 제한**: 개인정보처리자는 수집 목적에 필요한 최소한의 개인정보만 수집해야 하며, 정보주체는 필요한 최소한의 정보 외의 개인정보 수집에 동의하지 않을 수 있습니다.\n* **개인정보 제공**: 개인정보처리자는 정보주체의 동의를 받거나 법률에 따라 개인정보를 제3자에게 제공할 수 있습니다.\n* **민감정보 처리 제한**: 개인정보처리자는 사상·신념, 건강, 성생활 등 정보주체의 사생활을 현저히 침해할 우려가 있는 민감정보를 처리할 때에는 정보주체의 동의를 받아야 합니다.\n* **고유식별정보 처리 제한**: 개인정보처리자는 개인을 고유하게 구별하기 위한 고유식별정보(주민등록번호 등)를 처리할 때에는 정보주체의 동의를 받거나 법률에서 허용하는 경우에만 처리할 수 있습니다.\n* **주민등록번호 처리 제한**: 개인정보처리자는 법률에서 구체적으로 요구하거나 정보주체 또는 제3자의 급박한 생명, 신체, 재산의 이익을 위하여 필요한 경우를 제외하고는 주민등록번호를 처리할 수 없습니다.\n* **고정형 영상정보처리기기 설치·운영 제한**: 공개된 장소에 고정형 영상정보처리기기를 설치·운영하려는 경우에는 법률에서 허용하는 경우 또는 범죄 예방, 시설 안전 및 관리, 화재 예방 등 정당한 목적이 있어야 합니다.\n\n**개인정보 보호법은 개인정보 보호의 중요성을 강조하고, 개인정보처리자에게 개인정보를 안전하게 관리하고 처리할 책임을 부여합니다.** 정보주체는 자신의 개인정보가 어떻게 수집, 이용, 제공되는지 알 권리가 있으며